In [0]:
import pandas as pd
import joblib

In [0]:
model_path = "/Workspace/Users/arorapuneet1998@gmail.com/DMML_Assignment/Model_Training/linear_regression_churn_model.pkl"

lr_model = joblib.load(model_path)
print("Model loaded successfully.")

# encoder = joblib.load('encoder.pkl')

#### Preprocess new data

In [0]:
df = spark.table('dmml.datamart.customer_churn_processed')
df_new = df.toPandas()

In [0]:
customer_ids = df_new['CustomerID']

# Drop CustomerID from features
X_new_raw = df_new.drop(columns=['CustomerID'])

In [0]:
# ENcoding already applied in the preprocessing step 

# Scaling already done in the preprocessing step

In [0]:
print(X_new_raw)

In [0]:
X_new_raw.describe()

In [0]:
X_new_raw = X_new_raw.drop(columns=['SeniorCitizenFlag','LongTenureFlag'])

In [0]:
X_new_raw['ContractType_One Year']=X_new_raw['ContractType_one_year']
X_new_raw['ContractType_Two Year']=X_new_raw['ContractType_two_year']
X_new_raw['Gender_M']=X_new_raw['Gender_male']
X_new_raw['PaymentMethod_Credit Card']=X_new_raw['PaymentMethod_credit_card']
X_new_raw['PaymentMethod_Mailed Check']=X_new_raw['PaymentMethod_mailed_check']
X_new_raw['PaymentMethod_Electronic Check']=X_new_raw['PaymentMethod_mailed_check']

X_new_raw = X_new_raw.drop(columns=['ContractType_one_year'])
X_new_raw = X_new_raw.drop(columns=['ContractType_two_year'])
X_new_raw = X_new_raw.drop(columns=['Gender_male'])
X_new_raw = X_new_raw.drop(columns=['PaymentMethod_credit_card'])
X_new_raw = X_new_raw.drop(columns=['PaymentMethod_mailed_check'])



In [0]:
X_new_raw

In [0]:
# Predict
y_pred = lr_model.predict(X_new_raw)
y_pred_proba = lr_model.predict_proba(X_new_raw)[:, 1]

In [0]:
df_new['Churn_Prediction'] = y_pred
df_new['Churn_Probability'] = y_pred_proba

In [0]:
y_pred

In [0]:
df_new

In [0]:
df_new['Churn_Prediction'].nunique()

In [0]:
df_prediction = df_new[['CustomerID','Churn_Prediction','Churn_Probability']]
print(df_prediction)

In [0]:
df_prediction_spark = spark.createDataFrame(df_prediction)

df_prediction_spark = df_prediction_spark.withColumnRenamed('Churn_Probability','Churn_Confidence')

display(df_prediction_spark)

In [0]:
df_prediction_spark.printSchema()

In [0]:
df_prediction_spark.write.mode("overwrite").saveAsTable("dmml.datamart.inference_customer_churn_temp")

In [0]:
spark.sql("select * from dmml.datamart.inference_customer_churn_temp").display()